<a href="https://colab.research.google.com/github/anujatayal/Colab-Notebooks/blob/main/signet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip /content/sample_data/OfflineGenuine.zip

Archive:  /content/sample_data/OfflineGenuine.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/sample_data/OfflineGenuine.zip or
        /content/sample_data/OfflineGenuine.zip.zip, and cannot find /content/sample_data/OfflineGenuine.zip.ZIP, period.


In [ ]:
!cp "/content/sample_data/OfflineGenuine.zip" .

In [ ]:
!unzip OfflineGenuine.zip

Archive:  OfflineGenuine.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of OfflineGenuine.zip or
        OfflineGenuine.zip.zip, and cannot find OfflineGenuine.zip.ZIP, period.


In [ ]:
!ls OfflineGenuine.zip

OfflineGenuine.zip


In [ ]:
ls /content/sample_data/Forgeries/

0100003_10.png  0109001_7.png   0109007_9.png   0113001_6.png   0114008_4.png
0100003_11.png  0109001_8.png   0110005_10.png  0113001_7.png   0114008_5.png
0100003_12.png  0109001_9.png   0110005_11.png  0113001_8.png   0114008_6.png
0100003_1.png   0109002_10.png  0110005_12.png  0113001_9.png   0114008_7.png
0100003_2.png   0109002_11.png  0110005_1.png   0113003_10.png  0114008_8.png
0100003_3.png   0109002_1.png   0110005_2.png   0113003_11.png  0114008_9.png
0100003_4.png   0109002_2.png   0110005_3.png   0113003_12.png  0116001_10.png
0100003_5.png   0109002_3.png   0110005_4.png   0113003_1.png   0116001_11.png
0100003_6.png   0109002_4.png   0110005_5.png   0113003_2.png   0116001_12.png
0100003_7.png   0109002_5.png   0110005_6.png   0113003_3.png   0116001_3.png
0100003_8.png   0109002_6.png   0110005_7.png   0113003_4.png   0116001_4.png
0100003_9.png   0109002_7.png   0110005_8.png   0113003_5.png   0116001_5.png
0101004_10.png  0109002_8.png   0110005_9.png   0113003_6.png

In [ ]:
forge = []

a = b = 0 
path = os.listdir('/content/sample_data/Forgeries/')
path.sort(key = lambda x: x[5:7])

for i in range(10):
  y = []
  for j in range(b,len(path)):
    if(path[j][5:7] == str(i+1).zfill(2)):
        img = cv2.imread('/content/sample_data/Forgeries/'+path[j]+'')
        img  = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
        arr = np.asarray(img)
        arr = cv2.resize(arr,(155,220),interpolation= cv2.INTER_AREA)
        y.append(arr)
        a = a + 1
        #print(a)
        if(a==235):
          forge.append(np.asarray(y))
    else:
     # print('...............')
      forge.append(np.asarray(y))
      b = a
      break

In [ ]:
gen = []

a = b = 0 
path = os.listdir('/content/sample_data/Genuine/')
path.sort(key = lambda sample: sample[1:3])

for i in range(10):
  y = []
  for j in range(b,len(path)):
    if(path[j][1:3] == str(i+1).zfill(2)):
        img = cv2.imread('/content/sample_data/Genuine/'+path[j]+'')
        img  = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
        arr = np.asarray(img)
        arr = cv2.resize(arr,(155,220),interpolation= cv2.INTER_AREA)
        y.append(arr)
        a = a + 1
        #print(a)
        if(a==235):
          gen.append(np.asarray(y))
    else:
      #print('...............')
      gen.append(np.asarray(y))
      b = a
      break

In [ ]:
len(gen)

10

In [ ]:

X1 = []
X2 = []
X3 = []
for i in range(10):
  for j in range(len(gen[i])):
    for l in range(10):
      #y1 = []
      X1.append(gen[i][j])
      a = np.random.randint(0,len(gen[i])-1)
      X2.append(gen[i][a])
      k = np.random.randint(0,9)
      if(k == i):
        b = np.random.randint(0,len(forge[i])-1)
        X3.append(forge[i][b])
        #y1 = np.asarray(y1)
      else:
        b = np.random.randint(0,len(gen[k])-1)
        X3.append(gen[k][b])
        #y1 = np.asarray(y1)
      #x.append(y1)
X1 = np.asarray(X1).reshape((2350,155,220,1))
X2 = np.asarray(X2).reshape((2350,155,220,1))
X3 = np.asarray(X3).reshape((2350,155,220,1))

In [ ]:
import keras
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Lambda,Concatenate
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D, Add,Reshape
from keras.models import Model
from keras import regularizers
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.initializers import glorot_normal
#import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

In [ ]:
conv_activation = 'relu'
deep_activation = 'relu'

input_shape = (155, 220, 3)
num_classes = 10

model = Sequential()
model.add(Conv2D(96,kernel_size=(11, 11),strides=(1, 1),activation=conv_activation,input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(96, (3, 3), strides=(2, 2),activation=conv_activation))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(16, (16, 16), activation=conv_activation))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(16, (16, 16), activation=conv_activation))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

# model.add(Dense(100, activation=deep_activation))
model.add(Dense(100, activation=deep_activation))
model.add(Dense(50, activation=deep_activation))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])  

model.summary()

In [ ]:
m = emb_model((155,220,1))


ValueError: ignored

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(np.uint8(x[2][0]))
plt.show()